<div class='alert' style='background-color: #273333; color: #E5E7E8; padding:26px 26px; border-radius:15px; font-size:40px;'><B>ShowUp</B>forHealth - Feature Engineering</div><span style='color: #273333; padding:26px 26px; font-size:11px;'> Powered by <B>AutoNote</B> | Jan du Plessis Notebook - <B>Feature Engineering </b></span><div style='margin:4px 26px; color:#273333; font-size:17px;'>
</div>

1. Time Registered with the Practice 
2. Past No-shows
3. Cyclical Data Features 
4. Rota Type Categorization

In [33]:
# Importing default Libraries
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
import seaborn as sns
import warnings

# Hi-resolution Plots and Matplotlib inline
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

# Set the maximum number of rows and columns to be displayed
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
warnings.filterwarnings('ignore')

# "magic commands" to enable autoreload of your imported packages
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [34]:
RAW_DATA = '~/code/janduplessis883/data-showup/data/raw-data/'
WEATHER_DATA = '~/code/janduplessis883/data-showup/data/weather/weather.csv'
IMD_DATA = '~/code/janduplessis883/data-showup/data/imd-master/imd_master.csv'

OUTPUT_DATA = '~/code/janduplessis883/data-showup/data/output-data/'

# Time Registered with the Practice

**Length of registration** with the practice to be calculated in months.<BR>
Subtract **Registration date** from **Appointment date** in months.

In [35]:
data = pd.read_csv(f'{OUTPUT_DATA}training_data_no_features.csv')
data.head(2)

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg
0,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.0,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
1,10-May-22,14-Jul-22,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17 00:00:00,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.0,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712


In [25]:
def format_datetime_columms(df):
    date_list = ['Appointment booked date','Appointment date','Registration date']
    for date in date_list:
        df[date] = pd.to_datetime(df[date])
        
    return df

In [26]:
format_datetime_columms(data)

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg
0,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
1,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
2,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712
3,2022-05-12,2022-07-05,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589,2022-07-05 09:00:00,17.7,0.0,40,W8 6JQ,Female,2014-01-07,Other White,(XaG62) Main spoken language Portuguese,Current,0.00,0,0,0,0,0,0,0,51.496001,-0.193468,22698.0,0.519419,0.816996
4,2022-05-17,2022-07-05,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135,2022-07-05 10:00:00,18.5,0.0,70,SW3 3SQ,Male,2017-03-06,Indian or British Indian,(XaG5t) Main spoken language English,Current,0.39,0,1,0,1,1,0,0,51.490771,-0.168178,10169.0,0.540792,1.656309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026752,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47464358,2019-12-31 15:00:00,8.9,0.0,87,SW10 9NG,Male,2003-09-09,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.484323,-0.180246,20040.0,1.056940,0.579572
1026753,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47493202,2019-12-31 15:00:00,8.9,0.0,36,SW5 0DY,Male,2012-05-02,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,0,0,0,1,0,0,0,51.489257,-0.189864,14414.0,0.386273,1.348580
1026754,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47487575,2019-12-31 15:00:00,8.9,0.0,42,SW10 9AL,Female,2011-02-10,Chinese,(XaG5t) Main spoken language English,Current,0.08,0,0,0,0,0,0,0,51.488467,-0.191806,14414.0,0.287596,1.445016
1026755,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47489391,2019-12-31 15:00:00,8.9,0.0,51,SW5 9BY,Female,1976-11-26,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,1,1,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,1.629449


In [27]:
import numpy as np
import pandas as pd

def months_registered(df):
    # Calculate the difference between two dates
    df['delta'] = df['Appointment date'] - df['Registration date']
    
    # Convert Timedelta to months
    df['months_registered'] = df['delta'].dt.total_seconds() / (60*60*24*30.44)
    df['months_registered'] = np.ceil(df['months_registered'])
    
    # Drop the temporary 'delta' column
    df.drop(columns=['delta'], inplace=True)
    
    return df


In [28]:
months_registered(data)

,Appointment booked date,Appointment date,Appointment status,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg,months_registered
0,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,155.0
1,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,155.0
2,2022-05-10,2022-07-14,Finished,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,Male,2009-08-17,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,155.0
3,2022-05-12,2022-07-05,Finished,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589,2022-07-05 09:00:00,17.7,0.0,40,W8 6JQ,Female,2014-01-07,Other White,(XaG62) Main spoken language Portuguese,Current,0.00,0,0,0,0,0,0,0,51.496001,-0.193468,22698.0,0.519419,0.816996,102.0
4,2022-05-17,2022-07-05,Finished,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135,2022-07-05 10:00:00,18.5,0.0,70,SW3 3SQ,Male,2017-03-06,Indian or British Indian,(XaG5t) Main spoken language English,Current,0.39,0,1,0,1,1,0,0,51.490771,-0.168178,10169.0,0.540792,1.656309,64.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026752,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47464358,2019-12-31 15:00:00,8.9,0.0,87,SW10 9NG,Male,2003-09-09,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.484323,-0.180246,20040.0,1.056940,0.579572,196.0
1026753,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47493202,2019-12-31 15:00:00,8.9,0.0,36,SW5 0DY,Male,2012-05-02,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,0,0,0,1,0,0,0,51.489257,-0.189864,14414.0,0.386273,1.348580,92.0
1026754,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47487575,2019-12-31 15:00:00,8.9,0.0,42,SW10 9AL,Female,2011-02-10,Chinese,(XaG5t) Main spoken language English,Current,0.08,0,0,0,0,0,0,0,51.488467,-0.191806,14414.0,0.287596,1.445016,107.0
1026755,2019-12-31,2019-12-31,Finished,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47489391,2019-12-31 15:00:00,8.9,0.0,51,SW5 9BY,Female,1976-11-26,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,1,1,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,1.629449,518.0


# Past No-shows

Identify past No-shows from the current dataset.<BR>
This has a heighter likelyhood of future no-shows.

In [36]:
def encode_appointment_status(df):
    df['App_status_encoded'] = [0 if app_status == 'Did Not Attend' else 1 
                                     for app_status in df['Appointment status']]
    df = df.drop(columns='Appointment status')
    return df

In [42]:
def count_noshows(df):
    
        noshow = df[df['App_status_encoded'] == 0]

        # Group by Patient ID and count no-shows
        no_show_count = noshow.groupby('Patient ID')['App_status_encoded'].count().reset_index(name='No_shows')
        df = df.merge(no_show_count, how='left', on='Patient ID').fillna(0)
        
        return df

In [39]:
data = encode_appointment_status(data)

In [43]:
data = count_noshows(data)

# Rota Type Categorization

Categorize rotas into one of the following groups:
- **GP** - GP, Registrar, Sessions
- **Nurse** - any clinic type with Nurse
- **HCA** - any clinic type with the word HCA
- **ARRS** - Physhicians Associate, Physiotherapist, FCP, Clinical Pharmacist 

In [55]:
import re

def extract_rota_type(text):
    # HOW TO APPLY IT
    # Apply extract_role function and overwrite Rota type column
    # full_appointments['Rota type'] = full_appointments['Rota type'].apply(extract_rota_type)     
    role_map = {
    'GP': ['GP', 'Registrar', 'Urgent', 'Telephone', '111', 'FY2', 'F2', 'Extended Hours', 'GP Clinic', 'Session'],
    'Nurse': ['Nurse', 'Nurse Practitioner'], 
    'HCA': ['HCA','Health Care Assistant', 'Phlebotomy'],
    'ARRS': ['Pharmacist', 'Paramedic', 'Physiotherapist', 'Physicians Associate', 'ARRS', 'PCN'],
    }

    for role, patterns in role_map.items():
        for pattern in patterns:
            if re.search(pattern, text):
                return role
    return 'DROP'

In [56]:
def map_rota_type(df):
    df['Rota type'] = df['Rota type'].map(extract_rota_type)
    return(df)

In [57]:
map_rota_type(data)

,Appointment booked date,Appointment date,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg,App_status_encoded,No_shows
0,10-May-22,14-Jul-22,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",GP,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17 00:00:00,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,12.0
1,10-May-22,14-Jul-22,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",GP,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17 00:00:00,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,12.0
2,10-May-22,14-Jul-22,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",GP,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,12.0
3,12-May-22,05-Jul-22,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Nurse,19580589,2022-07-05 09:00:00,17.7,0.0,40,W8 6JQ,0,2014-01-07 00:00:00,Other White,(XaG62) Main spoken language Portuguese,Current,0.00,0,0,0,0,0,0,0,51.496001,-0.193468,22698.0,0.519419,0.816996,1,2.0
4,17-May-22,05-Jul-22,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Nurse,47551135,2022-07-05 10:00:00,18.5,0.0,70,SW3 3SQ,2,2017-03-06 00:00:00,Indian or British Indian,(XaG5t) Main spoken language English,Current,0.39,0,1,0,1,1,0,0,51.490771,-0.168178,10169.0,0.540792,1.656309,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026752,31-Dec-19,31-Dec-19,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",ARRS,47464358,2019-12-31 15:00:00,8.9,0.0,87,SW10 9NG,2,2003-09-09 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.484323,-0.180246,20040.0,1.056940,0.579572,1,22.0
1026753,31-Dec-19,31-Dec-19,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",ARRS,47493202,2019-12-31 15:00:00,8.9,0.0,36,SW5 0DY,2,2012-05-02 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,0,0,0,1,0,0,0,51.489257,-0.189864,14414.0,0.386273,1.348580,1,1.0
1026754,31-Dec-19,31-Dec-19,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",ARRS,47487575,2019-12-31 15:00:00,8.9,0.0,42,SW10 9AL,0,2011-02-10 00:00:00,Chinese,(XaG5t) Main spoken language English,Current,0.08,0,0,0,0,0,0,0,51.488467,-0.191806,14414.0,0.287596,1.445016,1,0.0
1026755,31-Dec-19,31-Dec-19,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",ARRS,47489391,2019-12-31 15:00:00,8.9,0.0,51,SW5 9BY,0,1976-11-26 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,1,1,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,1.629449,1,3.0


# Label Encode Sex

In [48]:
from sklearn.preprocessing import LabelEncoder

# Label Encode Sex
def labelencode_sex(df):
    le = LabelEncoder()
    df['Sex'] = le.fit_transform(df['Sex'])
    
    return df

In [49]:
labelencode_sex(data)

,Appointment booked date,Appointment date,Appointment time,Booked by,Clinician,Rota type,Patient ID,app datetime,temp,precipitation,Age in years,Postcode,Sex,Registration date,Ethnicity category,Language,Registration status,FRAILTY,DEPRESSION,OBESITY,IHD,DM,HPT,NDHG,SMI,Latitude,Longitude,IMD2023,dist_to_station,distance_from_surg,App_status_encoded,No_shows
0,10-May-22,14-Jul-22,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17 00:00:00,Other Mixed,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,12.0
1,10-May-22,14-Jul-22,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17 00:00:00,Other,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,12.0
2,10-May-22,14-Jul-22,10:00 - 10:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Registrar,19581842,2022-07-14 10:00:00,21.6,0.0,73,SW5 9BY,2,2009-08-17 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,0.204712,1,12.0
3,12-May-22,05-Jul-22,09:00 - 09:59,"FLORKO, Aleksandra (Miss)","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,19580589,2022-07-05 09:00:00,17.7,0.0,40,W8 6JQ,0,2014-01-07 00:00:00,Other White,(XaG62) Main spoken language Portuguese,Current,0.00,0,0,0,0,0,0,0,51.496001,-0.193468,22698.0,0.519419,0.816996,1,2.0
4,17-May-22,05-Jul-22,10:00 - 10:59,"LE TELLIER, Jean-Pierre","FLORKO, Aleksandra (Miss)",Earls Court Surgery Nurse,47551135,2022-07-05 10:00:00,18.5,0.0,70,SW3 3SQ,2,2017-03-06 00:00:00,Indian or British Indian,(XaG5t) Main spoken language English,Current,0.39,0,1,0,1,1,0,0,51.490771,-0.168178,10169.0,0.540792,1.656309,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1026752,31-Dec-19,31-Dec-19,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47464358,2019-12-31 15:00:00,8.9,0.0,87,SW10 9NG,2,2003-09-09 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,"Deceased, Deducted",0.00,0,0,0,0,0,0,0,51.484323,-0.180246,20040.0,1.056940,0.579572,1,22.0
1026753,31-Dec-19,31-Dec-19,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47493202,2019-12-31 15:00:00,8.9,0.0,36,SW5 0DY,2,2012-05-02 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,0,0,0,1,0,0,0,51.489257,-0.189864,14414.0,0.386273,1.348580,1,1.0
1026754,31-Dec-19,31-Dec-19,15:00 - 15:59,"AMADI, Kasi (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47487575,2019-12-31 15:00:00,8.9,0.0,42,SW10 9AL,0,2011-02-10 00:00:00,Chinese,(XaG5t) Main spoken language English,Current,0.08,0,0,0,0,0,0,0,51.488467,-0.191806,14414.0,0.287596,1.445016,1,0.0
1026755,31-Dec-19,31-Dec-19,15:00 - 15:59,"DUFFIELD, Katie (Miss)","PATEL, Rinku (Pharmacist)",Redcliffe Pharmacist,47489391,2019-12-31 15:00:00,8.9,0.0,51,SW5 9BY,0,1976-11-26 00:00:00,British or Mixed British,(XaG5t) Main spoken language English,Current,0.06,1,1,0,0,0,0,0,51.490204,-0.193625,16808.0,0.136631,1.629449,1,3.0
